In [19]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm
import gc
import warnings
from time import time
import xgboost as xgb
xgb.__version__

'1.6.0-dev'

In [20]:
optuna_xgb_output_100 = pd.read_csv('optuna_xgb_fe_output.csv')
optuna_xgb_output_100.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_num_class,params_objective,params_subsample,params_tree_method,state
0,0,0.956893,0.154753,0.938749,mlogloss,0.779656,0.001448,20,73,6,multi:softprob,0.483028,gpu_hist,COMPLETE
1,1,0.953298,0.030681,0.406860,mlogloss,8.650397,0.050611,3,59,6,multi:softprob,0.738625,gpu_hist,COMPLETE
2,2,0.929480,0.001916,0.868243,mlogloss,0.010653,0.001334,5,32,6,multi:softprob,0.459071,gpu_hist,COMPLETE
3,3,0.913466,0.072575,0.301956,mlogloss,0.307360,0.006653,3,245,6,multi:softprob,0.963456,gpu_hist,COMPLETE
4,4,0.962164,1.413603,0.904432,mlogloss,0.756485,0.037433,20,190,6,multi:softprob,0.564118,gpu_hist,COMPLETE


In [21]:
optuna_xgb_output_100.sort_values(by='value', ascending=False).head(1)

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_num_class,params_objective,params_subsample,params_tree_method,state
143,143,0.962738,0.005631,0.643151,mlogloss,3.121097,0.022424,11,17,6,multi:softprob,0.82267,gpu_hist,COMPLETE


In [22]:
params = {'objective': 'multi:softprob',
          'tree_method': 'gpu_hist',
          'lambda': 3.121097,
          'alpha': 0.005631,
          'colsample_bytree': 0.643151,
          'subsample': 0.82267,
          'learning_rate': 0.022424,
          'max_depth': 11,
          'min_child_weight': 17,
          'num_class':6,
          'eval_metric': 'merror'}

In [23]:
%%time
train = pd.read_csv('../../TPS_2021/input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('../../TPS_2021/input/tabular-playground-series-dec-2021/test.csv')

CPU times: user 6.3 s, sys: 719 ms, total: 7.02 s
Wall time: 7.02 s


In [24]:
train.drop(columns=['Soil_Type7', 'Soil_Type15', 'Id'], inplace=True) 
test.drop(columns=['Soil_Type7', 'Soil_Type15', 'Id'], inplace=True)

In [25]:
test.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,2763,78,20,377,88,3104,218,213,195,1931,...,0,0,0,0,0,0,0,0,0,0
1,2826,153,11,264,39,295,219,238,148,2557,...,0,0,0,0,0,0,0,0,0,0
2,2948,57,19,56,44,852,202,217,163,1803,...,0,0,1,0,0,0,0,0,0,0
3,2926,119,6,158,134,2136,234,240,142,857,...,0,0,0,0,0,0,0,0,0,0
4,2690,10,4,38,108,3589,213,221,229,431,...,0,0,0,0,0,0,0,0,0,0


In [26]:
train.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,3189,40,8,30,13,3270,206,234,193,4873,...,0,0,0,0,0,0,0,0,0,1
1,3026,182,5,280,29,3270,233,240,106,5423,...,0,0,0,0,0,0,0,0,0,2
2,3106,13,7,351,37,2914,208,234,137,5269,...,0,0,0,0,0,0,0,0,0,1
3,3022,276,13,192,16,3034,207,238,156,2866,...,0,0,0,0,0,0,0,0,0,2
4,2906,186,13,266,22,2916,231,231,154,2642,...,0,0,0,0,0,0,0,0,0,2


In [27]:
%%time
new_names = {
    "Horizontal_Distance_To_Hydrology": "x_dist_hydrlgy",
    "Vertical_Distance_To_Hydrology": "y_dist_hydrlgy",
    "Horizontal_Distance_To_Roadways": "x_dist_rdwys",
    "Horizontal_Distance_To_Fire_Points": "x_dist_firepts"
}

train.rename(new_names, axis=1, inplace=True)
test.rename(new_names, axis=1, inplace=True)

train["Aspect"][train["Aspect"] < 0] += 360
test["Aspect"][train["Aspect"] > 359] -= 360

test["Aspect"][test["Aspect"] < 0] += 360
test["Aspect"][test["Aspect"] > 359] -= 360

# Manhhattan distance to Hydrology
train["mnhttn_dist_hydrlgy"] = np.abs(train["x_dist_hydrlgy"]) + np.abs(train["y_dist_hydrlgy"])
test["mnhttn_dist_hydrlgy"] = np.abs(test["x_dist_hydrlgy"]) + np.abs(test["y_dist_hydrlgy"])

# Euclidean distance to Hydrology
train["ecldn_dist_hydrlgy"] = (train["x_dist_hydrlgy"]**2 + train["y_dist_hydrlgy"]**2)**0.5
test["ecldn_dist_hydrlgy"] = (test["x_dist_hydrlgy"]**2 + test["y_dist_hydrlgy"]**2)**0.5

train.loc[train["Hillshade_9am"] < 0, "Hillshade_9am"] = 0
test.loc[test["Hillshade_9am"] < 0, "Hillshade_9am"] = 0

train.loc[train["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0
test.loc[test["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0

train.loc[train["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0
test.loc[test["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0

train.loc[train["Hillshade_9am"] > 255, "Hillshade_9am"] = 255
test.loc[test["Hillshade_9am"] > 255, "Hillshade_9am"] = 255

train.loc[train["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255
test.loc[test["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255

train.loc[train["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255
test.loc[test["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255

features_Hillshade = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
soil_features = [x for x in train.columns if x.startswith("Soil_Type")]
wilderness_features = [x for x in train.columns if x.startswith("Wilderness_Area")]

def addFeature(X):
    # Thanks @mpwolke : https://www.kaggle.com/mpwolke/tooezy-where-are-you-no-camping-here
    X["Soil_Count"] = X[soil_features].apply(sum, axis=1)

    # Thanks @yannbarthelemy : https://www.kaggle.com/yannbarthelemy/tps-december-first-simple-feature-engineering
    X["Wilderness_Area_Count"] = X[wilderness_features].apply(sum, axis=1)
    X["Hillshade_mean"] = X[features_Hillshade].mean(axis=1)
    X['amp_Hillshade'] = X[features_Hillshade].max(axis=1) - X[features_Hillshade].min(axis=1)
    
addFeature(train)
addFeature(test)

cols = [
    "Elevation",
    "Aspect",
    "mnhttn_dist_hydrlgy",
    "ecldn_dist_hydrlgy",
    "Slope",
    "x_dist_hydrlgy",
    "y_dist_hydrlgy",
    "x_dist_rdwys",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "x_dist_firepts",
    
    "Soil_Count","Wilderness_Area_Count","Hillshade_mean","amp_Hillshade"
]

scaler = RobustScaler()
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])

features = test.columns

perm_dict = {1:2, 2:1, 3:3, 4:6, 7:4, 6:5, 5:7,}

train['Cover_Type'].replace(perm_dict, inplace=True)

train = train[train.Cover_Type !=7]

def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
 
    return df

train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to 438.69 Mb (76.0% reduction)
Mem. usage decreased to 101.09 Mb (77.2% reduction)
CPU times: user 1min 11s, sys: 8.08 s, total: 1min 19s
Wall time: 1min 20s


In [28]:
target = train[['Cover_Type']].values
train.drop(['Cover_Type'], axis=1, inplace=True)

In [29]:
target -= 1

In [30]:
np.unique(target)

array([0, 1, 2, 3, 4, 5], dtype=int8)

In [31]:
xgtest = xgb.DMatrix(test[features].values)

In [32]:
y_oof = np.zeros((train.shape[0], 6))
preds_total = 0

In [33]:
%%time
NFOLDS = 10
NSEEDS = 10
start_time = time()
kf = StratifiedKFold(NFOLDS, shuffle=True, random_state=137)
for i, (train_index, test_index) in enumerate(kf.split(train,target)):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[features].values[train_index], train[features].values[test_index]
        y_train, y_val = target[train_index], target[test_index]
        xgtrain = xgb.DMatrix(xgtrain, label = y_train ) 
        xgval = xgb.DMatrix(xgval, label = y_val ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        for j in range(NSEEDS):
            params['random_state'] = 224 + 17*j**3 +2*i**2
            clf = xgb.train(params, xgtrain, num_boost_round=1000, evals=watchlist, verbose_eval=1000)
            preds = clf.predict(xgtest)
            ypred = clf.predict(xgval)
            y_oof[test_index] += ypred/NSEEDS
            preds_total += preds/(NFOLDS*NSEEDS)
            del clf
            gc.collect()

        print('Time elapsed:', time()-start_time)
        #clf.save_model('xgb_gpu_hist_1812_2_fold_'+str(i)+'.model')


        print('Fold Accuracy:', accuracy_score(y_val, y_oof[test_index].argmax(axis=1)))
        
np.save('../metafeatures/train/y_oof_xgb_fe_1', y_oof)
np.save('../metafeatures/test/preds_total_xgb_fe_1', preds_total)
        

0
[0]	train-merror:0.08477	eval-merror:0.08587
[999]	train-merror:0.02815	eval-merror:0.03823
[0]	train-merror:0.06083	eval-merror:0.06196
[999]	train-merror:0.02812	eval-merror:0.03807
[0]	train-merror:0.08922	eval-merror:0.09018
[999]	train-merror:0.02824	eval-merror:0.03809
[0]	train-merror:0.07516	eval-merror:0.07578
[999]	train-merror:0.02812	eval-merror:0.03808
[0]	train-merror:0.06791	eval-merror:0.06852
[999]	train-merror:0.02807	eval-merror:0.03813
[0]	train-merror:0.08037	eval-merror:0.08094
[999]	train-merror:0.02812	eval-merror:0.03818
[0]	train-merror:0.36920	eval-merror:0.37010
[999]	train-merror:0.02815	eval-merror:0.03821
[0]	train-merror:0.07583	eval-merror:0.07636
[999]	train-merror:0.02806	eval-merror:0.03807
[0]	train-merror:0.06940	eval-merror:0.07052
[999]	train-merror:0.02811	eval-merror:0.03813
[0]	train-merror:0.08390	eval-merror:0.08514
[999]	train-merror:0.02809	eval-merror:0.03811
Time elapsed: 2938.6091549396515
Fold Accuracy: 0.9619475
1
[0]	train-merror:0

In [17]:
y_oof[test_index].argmax(axis=1)

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
print('OOF Accuracy:', accuracy_score(target, y_oof.argmax(axis=1)))

OOF Accuracy: 0.9623077405769351


In [19]:
y_oof.shape

(3999999, 6)

In [20]:
ypred.shape

(399999, 6)

In [21]:
preds_total.shape

(1000000, 6)

In [22]:
np.save('../metafeatures/train/y_oof_xgb_fe_1', y_oof)
np.save('../metafeatures/test/preds_total_xgb_fe_1', preds_total)

In [34]:
submission = pd.read_csv('../../TPS_2021/input/tabular-playground-series-dec-2021/sample_submission.csv')

In [35]:
inv_perm = {v: k for k, v in perm_dict.items()}
inv_perm

{2: 1, 1: 2, 3: 3, 6: 4, 4: 7, 5: 6, 7: 5}

In [36]:
submission['Cover_Type'] = preds_total.argmax(axis=1)+1
submission['Cover_Type'].replace(inv_perm, inplace=True)
submission.head(10)

,Id,Cover_Type
0,4000000,2
1,4000001,2
2,4000002,2
3,4000003,2
4,4000004,2
5,4000005,3
6,4000006,2
7,4000007,1
8,4000008,2
9,4000009,3


In [28]:
np.unique(submission['Cover_Type'].values)

array([1, 2, 3, 4, 6, 7])

In [37]:
submission.to_csv('../submissions/submission_xgb_best_optuna_fe_1.csv', index=False)

In [38]:
%%time
y_oof = np.zeros((train.shape[0], 6))
preds_total = 0
NFOLDS = 10
NSEEDS = 10
start_time = time()
kf = StratifiedKFold(NFOLDS, shuffle=True, random_state=137)
for i, (train_index, test_index) in enumerate(kf.split(train,target)):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[features].values[train_index], train[features].values[test_index]
        y_train, y_val = target[train_index], target[test_index]
        xgtrain = xgb.DMatrix(xgtrain, label = y_train ) 
        xgval = xgb.DMatrix(xgval, label = y_val ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        for j in range(NSEEDS):
            params['random_state'] = 225 + 17*j**3 +2*i**2
            clf = xgb.train(params, xgtrain, num_boost_round=900, evals=watchlist, verbose_eval=1000)
            preds = clf.predict(xgtest)
            ypred = clf.predict(xgval)
            y_oof[test_index] += ypred/NSEEDS
            preds_total += preds/(NFOLDS*NSEEDS)
            del clf
            gc.collect()

        print('Time elapsed:', time()-start_time)
        #clf.save_model('xgb_gpu_hist_1812_2_fold_'+str(i)+'.model')


        print('Fold Accuracy:', accuracy_score(y_val, y_oof[test_index].argmax(axis=1)))
        
np.save('../metafeatures/train/y_oof_xgb_fe_2', y_oof)
np.save('../metafeatures/test/preds_total_xgb_fe_2', preds_total)
        

submission['Cover_Type'] = preds_total.argmax(axis=1)+1
submission['Cover_Type'].replace(inv_perm, inplace=True)


submission.to_csv('../submissions/submission_xgb_best_optuna_fe_2.csv', index=False)

0
[0]	train-merror:0.08704	eval-merror:0.08738
[899]	train-merror:0.02891	eval-merror:0.03816
[0]	train-merror:0.05587	eval-merror:0.05680
[899]	train-merror:0.02899	eval-merror:0.03821
[0]	train-merror:0.05994	eval-merror:0.06177
[899]	train-merror:0.02895	eval-merror:0.03809
[0]	train-merror:0.07462	eval-merror:0.07573
[899]	train-merror:0.02892	eval-merror:0.03814
[0]	train-merror:0.06296	eval-merror:0.06437
[899]	train-merror:0.02896	eval-merror:0.03814
[0]	train-merror:0.06866	eval-merror:0.06906
[899]	train-merror:0.02896	eval-merror:0.03808
[0]	train-merror:0.37764	eval-merror:0.37884
[899]	train-merror:0.02888	eval-merror:0.03811
[0]	train-merror:0.39039	eval-merror:0.39065
[899]	train-merror:0.02898	eval-merror:0.03810
[0]	train-merror:0.06579	eval-merror:0.06660
[899]	train-merror:0.02894	eval-merror:0.03810
[0]	train-merror:0.10254	eval-merror:0.10307
[899]	train-merror:0.02901	eval-merror:0.03814
Time elapsed: 2668.161721229553
Fold Accuracy: 0.9619525
1
[0]	train-merror:0.

In [39]:
print('OOF Accuracy:', accuracy_score(target, y_oof.argmax(axis=1)))

OOF Accuracy: 0.9623129905782476


In [40]:
%%time
y_oof = np.zeros((train.shape[0], 6))
preds_total = 0
NFOLDS = 10
NSEEDS = 10
start_time = time()
kf = StratifiedKFold(NFOLDS, shuffle=True, random_state=137)
for i, (train_index, test_index) in enumerate(kf.split(train,target)):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[features].values[train_index], train[features].values[test_index]
        y_train, y_val = target[train_index], target[test_index]
        xgtrain = xgb.DMatrix(xgtrain, label = y_train ) 
        xgval = xgb.DMatrix(xgval, label = y_val ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        for j in range(NSEEDS):
            params['random_state'] = 229 + 17*j**3 +2*i**2
            clf = xgb.train(params, xgtrain, num_boost_round=1100, evals=watchlist, verbose_eval=1000)
            preds = clf.predict(xgtest)
            ypred = clf.predict(xgval)
            y_oof[test_index] += ypred/NSEEDS
            preds_total += preds/(NFOLDS*NSEEDS)
            del clf
            gc.collect()

        print('Time elapsed:', time()-start_time)
        #clf.save_model('xgb_gpu_hist_1812_2_fold_'+str(i)+'.model')


        print('Fold Accuracy:', accuracy_score(y_val, y_oof[test_index].argmax(axis=1)))
        
np.save('../metafeatures/train/y_oof_xgb_fe_3', y_oof)
np.save('../metafeatures/test/preds_total_xgb_fe_3', preds_total)
        

submission['Cover_Type'] = preds_total.argmax(axis=1)+1
submission['Cover_Type'].replace(inv_perm, inplace=True)


submission.to_csv('../submissions/submission_xgb_best_optuna_fe_3.csv', index=False)
print('OOF Accuracy:', accuracy_score(target, y_oof.argmax(axis=1)))

0
[0]	train-merror:0.37752	eval-merror:0.37789
[1000]	train-merror:0.02816	eval-merror:0.03803
[1099]	train-merror:0.02740	eval-merror:0.03804
[0]	train-merror:0.07249	eval-merror:0.07319
[1000]	train-merror:0.02816	eval-merror:0.03807
[1099]	train-merror:0.02739	eval-merror:0.03805
[0]	train-merror:0.07873	eval-merror:0.07931
[1000]	train-merror:0.02816	eval-merror:0.03809
[1099]	train-merror:0.02739	eval-merror:0.03810
[0]	train-merror:0.06496	eval-merror:0.06661
[1000]	train-merror:0.02814	eval-merror:0.03812
[1099]	train-merror:0.02740	eval-merror:0.03815
[0]	train-merror:0.09179	eval-merror:0.09194
[1000]	train-merror:0.02812	eval-merror:0.03812
[1099]	train-merror:0.02735	eval-merror:0.03802
[0]	train-merror:0.07388	eval-merror:0.07475
[1000]	train-merror:0.02816	eval-merror:0.03810
[1099]	train-merror:0.02736	eval-merror:0.03806
[0]	train-merror:0.08357	eval-merror:0.08448
[1000]	train-merror:0.02814	eval-merror:0.03812
[1099]	train-merror:0.02735	eval-merror:0.03814
[0]	train-m

In [34]:
print('OOF Accuracy:', accuracy_score(target, y_oof.argmax(axis=1)))

OOF Accuracy: 0.9623142405785602


In [41]:
%%time
y_oof = np.zeros((train.shape[0], 6))
preds_total = 0
NFOLDS = 10
NSEEDS = 10
start_time = time()
kf = StratifiedKFold(NFOLDS, shuffle=True, random_state=137)
for i, (train_index, test_index) in enumerate(kf.split(train,target)):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[features].values[train_index], train[features].values[test_index]
        y_train, y_val = target[train_index], target[test_index]
        xgtrain = xgb.DMatrix(xgtrain, label = y_train ) 
        xgval = xgb.DMatrix(xgval, label = y_val ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        for j in range(NSEEDS):
            params['random_state'] = 233 + 17*j**3 +2*i**2
            clf = xgb.train(params, xgtrain, num_boost_round=1200, evals=watchlist, verbose_eval=1000)
            preds = clf.predict(xgtest)
            ypred = clf.predict(xgval)
            y_oof[test_index] += ypred/NSEEDS
            preds_total += preds/(NFOLDS*NSEEDS)
            del clf
            gc.collect()

        print('Time elapsed:', time()-start_time)
        #clf.save_model('xgb_gpu_hist_1812_2_fold_'+str(i)+'.model')


        print('Fold Accuracy:', accuracy_score(y_val, y_oof[test_index].argmax(axis=1)))
        
np.save('../metafeatures/train/y_oof_xgb_fe_4', y_oof)
np.save('../metafeatures/test/preds_total_xgb_fe_4', preds_total)
        

submission['Cover_Type'] = preds_total.argmax(axis=1)+1
submission['Cover_Type'].replace(inv_perm, inplace=True)


submission.to_csv('../submissions/submission_xgb_best_optuna_fe_4.csv', index=False)
print('OOF Accuracy:', accuracy_score(target, y_oof.argmax(axis=1)))

0
[0]	train-merror:0.09104	eval-merror:0.09246
[1000]	train-merror:0.02803	eval-merror:0.03822
[1199]	train-merror:0.02648	eval-merror:0.03819
[0]	train-merror:0.06418	eval-merror:0.06473
[1000]	train-merror:0.02815	eval-merror:0.03810
[1199]	train-merror:0.02656	eval-merror:0.03802
[0]	train-merror:0.08243	eval-merror:0.08331
[1000]	train-merror:0.02813	eval-merror:0.03816
[1199]	train-merror:0.02656	eval-merror:0.03813
[0]	train-merror:0.05646	eval-merror:0.05745
[1000]	train-merror:0.02812	eval-merror:0.03814
[1199]	train-merror:0.02654	eval-merror:0.03804
[0]	train-merror:0.06560	eval-merror:0.06644
[1000]	train-merror:0.02807	eval-merror:0.03814
[1199]	train-merror:0.02654	eval-merror:0.03804
[0]	train-merror:0.06409	eval-merror:0.06484
[1000]	train-merror:0.02818	eval-merror:0.03810
[1199]	train-merror:0.02666	eval-merror:0.03808
[0]	train-merror:0.06838	eval-merror:0.06906
[1000]	train-merror:0.02811	eval-merror:0.03812
[1199]	train-merror:0.02651	eval-merror:0.03803
[0]	train-m

In [36]:
print('OOF Accuracy:', accuracy_score(target, y_oof.argmax(axis=1)))

OOF Accuracy: 0.9623114905778727


In [42]:
%%time
y_oof = np.zeros((train.shape[0], 6))
preds_total = 0
NFOLDS = 10
NSEEDS = 10
start_time = time()
kf = StratifiedKFold(NFOLDS, shuffle=True, random_state=137)
for i, (train_index, test_index) in enumerate(kf.split(train,target)):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[features].values[train_index], train[features].values[test_index]
        y_train, y_val = target[train_index], target[test_index]
        xgtrain = xgb.DMatrix(xgtrain, label = y_train ) 
        xgval = xgb.DMatrix(xgval, label = y_val ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        for j in range(NSEEDS):
            params['random_state'] = 237 + 17*j**3 +2*i**2
            clf = xgb.train(params, xgtrain, num_boost_round=1300, evals=watchlist, verbose_eval=1000)
            preds = clf.predict(xgtest)
            ypred = clf.predict(xgval)
            y_oof[test_index] += ypred/NSEEDS
            preds_total += preds/(NFOLDS*NSEEDS)
            del clf
            gc.collect()

        print('Time elapsed:', time()-start_time)
        #clf.save_model('xgb_gpu_hist_1812_2_fold_'+str(i)+'.model')


        print('Fold Accuracy:', accuracy_score(y_val, y_oof[test_index].argmax(axis=1)))
        
np.save('../metafeatures/train/y_oof_xgb_fe_5', y_oof)
np.save('../metafeatures/test/preds_total_xgb_fe_5', preds_total)
        

submission['Cover_Type'] = preds_total.argmax(axis=1)+1
submission['Cover_Type'].replace(inv_perm, inplace=True)


submission.to_csv('../submissions/submission_xgb_best_optuna_fe_5.csv', index=False)
print('OOF Accuracy:', accuracy_score(target, y_oof.argmax(axis=1)))

0
[0]	train-merror:0.08192	eval-merror:0.08250
[1000]	train-merror:0.02814	eval-merror:0.03816
[1299]	train-merror:0.02576	eval-merror:0.03816
[0]	train-merror:0.06115	eval-merror:0.06214
[1000]	train-merror:0.02814	eval-merror:0.03812
[1299]	train-merror:0.02576	eval-merror:0.03805
[0]	train-merror:0.05992	eval-merror:0.06072
[1000]	train-merror:0.02812	eval-merror:0.03803
[1299]	train-merror:0.02578	eval-merror:0.03811
[0]	train-merror:0.06093	eval-merror:0.06243
[1000]	train-merror:0.02818	eval-merror:0.03803
[1299]	train-merror:0.02584	eval-merror:0.03807
[0]	train-merror:0.06412	eval-merror:0.06545
[1000]	train-merror:0.02820	eval-merror:0.03805
[1299]	train-merror:0.02583	eval-merror:0.03799
[0]	train-merror:0.39358	eval-merror:0.39384
[1000]	train-merror:0.02819	eval-merror:0.03804
[1299]	train-merror:0.02584	eval-merror:0.03807
[0]	train-merror:0.08254	eval-merror:0.08386
[1000]	train-merror:0.02815	eval-merror:0.03811
[1299]	train-merror:0.02578	eval-merror:0.03802
[0]	train-m

In [38]:
print('OOF Accuracy:', accuracy_score(target, y_oof.argmax(axis=1)))

OOF Accuracy: 0.9623184905796226
